In [1]:
using CSV
using DataFrames
using JuMP
using Plots
using Random
using Statistics
using LinearAlgebra
using Distributions
using BipartiteMatching
using Gurobi
using LinearAlgebra
using SymPy
using NLsolve
using LaTeXStrings


# Proof of Theorem 4

### Goal
The notebook aims to verify if $$1-(1-B) \cdot e^{-0.01} < (1-(B-\delta)/2) \phi(B-\delta) + (B-\delta) - 0.0001$$ within each set $[B, B + \delta) \subseteq [0.4,0.8]$. That is, we verify $$(1-B) \cdot e^{-0.01} + (1-(B-\delta)/2) \phi(B-\delta) - 1 + (B-\delta) - 0.0001 > 0$$ within each set $[B, B + \delta) \subseteq [0.4,0.8]$.

### Arguments
- `x_lb` `x_ub`: The respective lower and upper bounds of $x$.
- `alphaf` or `alphaf_val`: Value of $\alpha^f$.
- `alpha` or `alpha_val`: Value of $\alpha$.
- `delta` or `delta_val`: Small positive increment.

### Functions
- `equation_x!(F, x, alphaf_val, alpha_val)` returns the numerical solutions of $x, x_1$ and $x_2$. `lb_comparison` uses these values to build the lower bound $\phi(B-\delta)$ for given $B$.
- `comp_f_func(alphaf_val, alpha_val)` returns the value of lb_comparison.
- `calculate_lb_comparison_matrix(delta_val)` examines all B values within the claimed region.

### Outputs
- `calculate_lb_comparison_matrix(delta_val)` should print nothing if all values of B pass the verification. This is indeed observed when delta = 0.0001.

In [58]:
#### Define an expression to compute the lower bound of mu(1,0)
@vars alphaf alpha x_lb x_ub
lb_comparison = 2-x_ub-exp(-(alphaf+alpha)*x_lb)*(1+(alphaf+alpha)*x_ub)

# Define a Julia function to solve x.
function equation_x!(F, x, alphaf_val, alpha_val)
    F[1] = exp(-(alphaf_val + alpha_val)*x[1]) - x[1]
end

# Define a Julia function to compute phi.
tolerance = 1e-8
function comp_f_func(alphaf_val, alpha_val)
    x_sol = nlsolve((F, x) -> equation_x!(F, x, alphaf_val, alpha_val), [0.2], autodiff=:forward, ftol=tolerance).zero[1]
    x_sol_ub, x_sol_lb = x_sol+tolerance, x_sol-tolerance
    lb_comparison_val = subs(lb_comparison, (alphaf, alphaf_val), (alpha, alpha_val), 
        (x_lb, x_sol_lb), (x_ub, x_sol_ub))
    return lb_comparison_val
end

# This function iterates over B, printing out cases where the inequality is not satisfied.
function calculate_lb_comparison_matrix(delta_val)
    
    budget_range = 0.4:delta_val:0.8

    for (i, budget) in enumerate(budget_range)
        alphaf_val = 0.009*(1-(budget-delta_val)/2)
        alpha_val = 0.0009*(1-(budget-delta_val)/2)
        comparison = (1-budget)*exp(-0.01) + (1-(budget-delta_val)/2)*comp_f_func(alphaf_val, alpha_val) - 1 + (budget-delta_val) - 0.0001
        if comparison <= 0
            println(budget, "\t", comparison)
        end
    end
    
    return 
end


calculate_lb_comparison_matrix (generic function with 1 method)

In [59]:
# delta = 0.0001: runtime ~ 1 min
delta_val = 0.0001
results_matrix = calculate_lb_comparison_matrix(delta_val)